In [58]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 24: Lobby Layout ---</h2><p>Your raft makes it to the tropical island; it turns out that the small crab was an excellent navigator. You make your way to the resort.</p>
<p>As you enter the lobby, you discover a small problem: the floor is being renovated. You can't even reach the check-in desk until they've finished installing the <em>new tile floor</em>.</p>
<p>The tiles are all <em>hexagonal</em>; they need to be arranged in a <a href="https://en.wikipedia.org/wiki/Hexagonal_tiling">hex grid</a> with a very specific color pattern. Not in the mood to wait, you offer to help figure out the pattern.</p>
<p>The tiles are all <em>white</em> on one side and <em>black</em> on the other. They start with the white side facing up. The lobby is large enough to fit whatever pattern might need to appear there.</p>
<p>A member of the renovation crew gives you a <em>list of the tiles that need to be flipped over</em> (your puzzle input). Each line in the list identifies a single tile that needs to be flipped by giving a series of steps starting from a <em>reference tile</em> in the very center of the room. (Every line starts from the same reference tile.)</p>
<p>Because the tiles are hexagonal, every tile has <em>six neighbors</em>: east, southeast, southwest, west, northwest, and northeast. These directions are given in your list, respectively, as <code>e</code>, <code>se</code>, <code>sw</code>, <code>w</code>, <code>nw</code>, and <code>ne</code>. A tile is identified by a series of these directions with <em>no delimiters</em>; for example, <code>esenee</code> identifies the tile you land on if you start at the reference tile and then move one tile east, one tile southeast, one tile northeast, and one tile east.</p>
<p>Each time a tile is identified, it flips from white to black or from black to white. Tiles might be flipped more than once. For example, a line like <code>esew</code> flips a tile immediately adjacent to the reference tile, and a line like <code>nwwswee</code> flips the reference tile itself.</p>
<p>Here is a larger example:</p>
<pre><code>sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
</code></pre>
<p>In the above example, 10 tiles are flipped once (to black), and 5 more are flipped twice (to black, then back to white). After all of these instructions have been followed, a total of <em><code>10</code></em> tiles are <em>black</em>.</p>
<p>Go through the renovation crew's list and determine which tiles they need to flip. After all of the instructions have been followed, <em>how many tiles are left with the black side up?</em></p>
</article>


In [59]:
example_small = "nwwswee"
example_big = """
sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
"""

DIRECTIONS = {
    "e": (1, 0, -1),
    "se": (1, -1, 0),
    "sw": (0, -1, 1),
    "w": (-1, 0, 1),
    "nw": (-1, 1, 0),
    "ne": (0, 1, -1),
}


def follow_directions(s: str) -> set[tuple[int, int, int]]:
    tiles = set()
    for instruction in s.strip().splitlines():
        x, y, z = 0, 0, 0
        for direction in re.findall(r"se|sw|nw|ne|e|w", instruction):
            dx, dy, dz = DIRECTIONS[direction]
            x += dx
            y += dy
            z += dz
        if (x, y, z) in tiles:
            tiles.remove((x, y, z))
        else:
            tiles.add((x, y, z))
    return tiles


def flip_tiles(s: str) -> int:
    tiles = follow_directions(s)
    return len(tiles)


assert flip_tiles(example_small) == 1
assert flip_tiles(example_big) == 10

In [60]:
with open("../input/day24.txt") as f:
    puzzle = f.read()

print(f"Part I: {flip_tiles(puzzle)}")

Part I: 434


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>434</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The tile floor in the lobby is meant to be a <span title="I need one of these!">living art exhibit</span>. Every day, the tiles are all flipped according to the following rules:</p>
<ul>
<li>Any <em>black</em> tile with <em>zero</em> or <em>more than 2</em> black tiles immediately adjacent to it is flipped to <em>white</em>.</li>
<li>Any <em>white</em> tile with <em>exactly 2</em> black tiles immediately adjacent to it is flipped to <em>black</em>.</li>
</ul>
<p>Here, <em>tiles immediately adjacent</em> means the six tiles directly touching the tile in question.</p>
<p>The rules are applied <em>simultaneously</em> to every tile; put another way, it is first determined which tiles need to be flipped, then they are all flipped at the same time.</p>
<p>In the above example, the number of black tiles that are facing up after the given number of days has passed is as follows:</p>
<pre><code>Day 1: 15
Day 2: 12
Day 3: 25
Day 4: 14
Day 5: 23
Day 6: 28
Day 7: 41
Day 8: 37
Day 9: 49
Day 10: 37

Day 20: 132
Day 30: 259
Day 40: 406
Day 50: 566
Day 60: 788
Day 70: 1106
Day 80: 1373
Day 90: 1844
Day 100: 2208
</code></pre>

<p>After executing this process a total of 100 times, there would be <em><code>2208</code></em> black tiles facing up.</p>
<p><em>How many tiles will be black after 100 days?</em></p>
</article>

</main>


In [61]:
def black_tiles(s: str, do_print=False) -> int:
    tiles = follow_directions(s)

    for day in range(1, 101):
        new_tiles = set()
        white_tiles = defaultdict(int)

        for x, y, z in tiles:
            # Any black tile with zero or more than 2 black tiles immediately
            # adjacent to it is flipped to white.
            black_neigbours = 0
            for dx, dy, dz in DIRECTIONS.values():
                if (x + dx, y + dy, z + dz) in tiles:
                    black_neigbours += 1
                else:
                    white_tiles[(x + dx, y + dy, z + dz)] += 1

            if not (black_neigbours == 0 or black_neigbours > 2):
                new_tiles.add((x, y, z))

        # Any white tile with exactly 2 black tiles immediately adjacent to it is flipped to black.
        new_tiles |= {
            white_tile for white_tile, count in white_tiles.items() if count == 2
        }

        tiles = new_tiles

        if do_print:
            print(f"Day {day}: {len(tiles)}")

    return len(tiles)


assert black_tiles(example_big) == 2208

In [62]:
print(f"Part II: {black_tiles(puzzle)}")

Part II: 3955


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3955</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
